In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# disable Altair row limit (important for world data)
alt.data_transformers.disable_max_rows()

# world map topojson (no installs needed)
WORLD_GEOJSON_URL = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"

In [14]:
# Load each file
war = pd.read_csv("geomap_war.csv")
rights = pd.read_csv("geomap_rights.csv")
freedom = pd.read_csv("geomap_freedom.csv")

# Identify the country column (Google Trends uses 'geoName' or 'Country')
country_col = [c for c in war.columns if "geo" in c.lower() or "country" in c.lower()][0]

# Keep country and interest columns
war = war[[country_col, war.columns[-1]]].rename(columns={country_col: "country", war.columns[-1]: "War"})
rights = rights[[country_col, rights.columns[-1]]].rename(columns={country_col: "country", rights.columns[-1]: "Rights"})
freedom = freedom[[country_col, freedom.columns[-1]]].rename(columns={country_col: "country", freedom.columns[-1]: "Freedom"})

# Combine all three on 'country'
search = war.merge(rights, on="country", how="outer").merge(freedom, on="country", how="outer")
search.head()


country   War  Rights Freedom
0     Afghanistan  11.0    12.0       1
1         Albania  14.0     9.0       1
2         Algeria   7.0     2.0       1
3  American Samoa  48.0    81.0       7
4         Andorra   8.0     4.0       1

In [20]:
import pandas as pd

# First, see what the top 10 rows look like:
acled = pd.read_excel("number_of_political_violence_events_by_country-month-year_as-of-03Oct2025.xlsx", nrows=10)
display(acled.head(10))


COUNTRY      MONTH  YEAR  EVENTS
0  Afghanistan    January  2017     865
1  Afghanistan   February  2017     697
2  Afghanistan      March  2017    1166
3  Afghanistan      April  2017    1079
4  Afghanistan        May  2017    1229
5  Afghanistan       June  2017     904
6  Afghanistan       July  2017    1281
7  Afghanistan     August  2017    1321
8  Afghanistan  September  2017     943
9  Afghanistan    October  2017    1228

**Data Sources**

In [33]:

# Filter to last 5 years
recent = acled[acled["YEAR"] >= acled["YEAR"].max() - 5]

# Aggregate total events by country
events = recent.groupby("COUNTRY", as_index=False)["EVENTS"].sum()
events = events.rename(columns={"EVENTS": "EVENTS"})

# Clean up country names (remove duplicates/extra spaces)
events["COUNTRY"] = events["COUNTRY"].astype(str).str.strip()

data = search.merge(events, on="country", how="left").fillna(0)

display(data.head())


NameError: name 'EVENTS' is not defined

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# data should have columns: ['country', 'War', 'Rights', 'Freedom', 'Events']

keywords = ["War", "Rights", "Freedom"]
colorscale = "Viridis"   # try 'Turbo', 'Reds', 'Plasma', 'Cividis', etc.

fig = go.Figure()

for i, kw in enumerate(keywords):
    fig.add_trace(go.Choropleth(
        locations = data["country"],
        z = data[kw],
        locationmode = "country names",
        colorscale = colorscale,
        zmin = 0, zmax = 100,
        colorbar_title = "Search Interest",
        visible = (i == 0),  # show only the first trace at start
        marker_line_color = "black",
        marker_line_width = 0.3,
        customdata = data[["Events"]],
        hovertemplate = (
            "<b>%{location}</b><br>" +
            kw + " interest: %{z:.0f}<br>" +
            "ACLED Events (5y): %{customdata[0]:,}<extra></extra>"
        ),
        name = kw
    ))

# Dropdown menu to switch keywords
buttons = []
for i, kw in enumerate(keywords):
    vis = [False] * len(keywords)
    vis[i] = True
    buttons.append(dict(
        label = kw,
        method = "update",
        args = [
            {"visible": vis},
            {"title": f"Global Search Interest: {kw} (Last 5 Years)"}
        ]
    ))

fig.update_layout(
    title = "Global Search Interest: War (Last 5 Years)",
    geo = dict(
        projection_type = "natural earth",
        showframe = False,
        showcoastlines = True,
        coastlinecolor = "rgba(0,0,0,0.5)",
        landcolor = "rgb(240,240,240)"
    ),
    updatemenus = [dict(
        type = "dropdown",
        x = 0.02, y = 1.05,
        xanchor = "left",
        buttons = buttons,
        showactive = True
    )],
    margin = dict(l=0, r=0, t=60, b=0)
)

fig.show()


In [ ]:
import altair as alt
from vega_datasets import data


variable_list = ['population', 'engineers', 'hurricanes']

alt.Chart(states).mark_geoshape().encode(
    color='variable_list:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(pop, 'id', list(pop.columns))
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)


NameError: name 'pop' is not defined

In [ ]:
import altair as alt

# Assumes you already have a DataFrame called `data` with columns:
# ['country', 'War', 'Rights', 'Freedom', 'Events']

alt.data_transformers.disable_max_rows()

WORLD_GEOJSON_URL = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
metrics = ["War", "Rights", "Freedom"]

# Dropdown (like your variable_list concept)
metric_sel = alt.selection_point(
    fields=["Metric"],
    bind=alt.binding_select(options=metrics, name="Search metric: "),
    value="War"
)

chart = (
    alt.Chart(alt.Data(url=WORLD_GEOJSON_URL, format={"type": "json", "property": "features"}))
    .mark_geoshape(stroke="white", strokeWidth=0.25)
    # ← Join your tabular data to the geoshapes by country name
    .transform_lookup(
        lookup="properties.name",                           # country name in GeoJSON
        from_=alt.LookupData(
            data,                                          # your dataframe
            key="country",                                 # join key
            fields=metrics + ["Events"]                    # bring in these columns
        )
    )
    # Fold (War/Rights/Freedom) so we can choose one via the dropdown
    .transform_fold(
        metrics,
        as_=["Metric", "Value"]
    )
    .transform_filter(metric_sel)
    .encode(
        
        color=alt.Color(
            "Value:Q",
            title="Search Interest (0–100)",
            scale=alt.Scale(scheme="redyellowblue", domain=[0, 100])  # bolder palette & full range
        ),
        tooltip=[
            alt.Tooltip("properties.name:N", title="Country"),
            alt.Tooltip("Metric:N"),
            alt.Tooltip("Value:Q", title="Interest", format=".0f"),
            alt.Tooltip("Events:Q", title="ACLED Events (5y)", format=",")
        ]
    )
    .project(type="mercator")
    .properties(
        width=900,
        height=200,
        title="Global Search Interest (War/Rights/Freedom) vs ACLED Events (Last 5 Years)"
    )
    .add_params(metric_sel)
)

chart


NameError: name 'data' is not defined